In [7]:
import os
import numpy as np
import pandas as pd

# load data '../../chars.csv'
chars = pd.read_csv('../characters_freq.csv', sep=',', encoding='utf-8')
words = pd.read_csv('../vocabulary_freq.csv', sep=',', encoding='utf-8')

In [8]:
# new dataframe
df = pd.DataFrame(columns=['page', 'chars', 'words', 'level', 'pinyin'])

page = 0
for index, row in chars.iterrows():

    page += 1

    df.loc[index, 'page'] = page
    df.loc[index, 'chars'] = row['character']
    df.loc[index, 'level'] = row['level']
    df.loc[index, 'pinyin'] = row['pinyin']

In [9]:
df['old_chars'] = ''
for page, row in df.iterrows():
    cum_chars = df.loc[:page, 'chars'].str.cat(sep=', ')
    df.at[page, 'old_chars'] = cum_chars

In [10]:
def find_words(df, words, num_chars_interval=(2, 10), max_page=-1):
    for page, row in df.iterrows():
        chars = row['old_chars']
        contains = words['word'].apply(lambda x: all(
            char in chars for char in x) and num_chars_interval[0] <= len(x) <= num_chars_interval[1])

        df.at[page, 'words'] = words[contains]['word'].values.tolist()
        # remove duplicates
        df.at[page, 'words'] = list(set(df.at[page, 'words']))

        # print(
        #     f"Page {row['page']}:\n\tChars: {chars}\n\tWords: {len(df.at[page, 'words'])}\n")

        # stop at max_page
        if row['page'] == max_page:
            break

find_words(df, words, num_chars_interval=(2, 10))

In [ ]:
# Define the order
level_order = ['一级', '二级', '三级', '四级', '五级', '六级', '高等']

# Sort the dataframe
df['level'] = pd.Categorical(df['level'], level_order)
df.sort_values(['level', 'page'], inplace=True)

# Save to csv
# df.to_csv('guide.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
# save every level (as numbers) to csv
for i, level in enumerate(level_order, 1):
    df[df['level'] == level].to_csv(
        f'guide_{i}.csv', sep=',', encoding='utf-8', index=False)